In [81]:
%import_ext autoreload
%autoreload 2

UsageError: Line magic function `%import_ext` not found.


In [82]:
import hashlib
import pandas as pd
from astropy.table import Table
import numpy as np
from sklearn.model_selection import train_test_split

In [83]:
df = Table.read('/nvme1/scratch/walml/repos/pytorch-galaxy-datasets/hidden_data/gz_hst_ids_assets_flags_withURLs.fits').to_pandas()

for str_col in ['zooniverse_id', 'location']:
    df[str_col] = df[str_col].apply(lambda x: x.decode('utf-8'))

df['location'] = df['location'].str.rstrip()  # often has trailing whitespace, I assume to be standard length

In [84]:
df

,asset_id,name,zooniverse_id,RA,Dec,cutout_size,location,OBJNO,SURVEY_ID,IMAGING,...,sample_cosmos_flagmissed,sample_fake_agn_v1_flagmissed,sample_cosmos_test,sample_cosmos_initial_test,sample_cosmos_flagmissed_test,sample_cosmos_inc_test,sample_cosmos_inc_test_initial,sample_cosmos_inc_test_flagmissed,sample,duplicate
0,1,10000189,AHZ1000001,214.235824,52.396332,b'120 ',http://zoo-hst.s3.amazonaws.com/10000189.jpg,10000189,-999,b'AEGIS ',...,False,False,False,False,False,False,False,False,b'original_initial ',False
1,2,10000215,AHZ1000002,214.328535,52.407050,b'185 ',http://zoo-hst.s3.amazonaws.com/10000215.jpg,10000215,-999,b'AEGIS ',...,False,False,False,False,False,False,False,False,b'original_initial ',False
2,3,10000235,AHZ1000003,214.130396,52.452309,b'182 ',http://zoo-hst.s3.amazonaws.com/10000235.jpg,10000235,-999,b'AEGIS ',...,False,False,False,False,False,False,False,False,b'original_initial ',False
3,4,10000249,AHZ1000004,214.225495,52.408517,b'120 ',http://zoo-hst.s3.amazonaws.com/10000249.jpg,10000249,-999,b'AEGIS ',...,False,False,False,False,False,False,False,False,b'original_initial ',False
4,5,10000278,AHZ1000005,214.257599,52.450259,b'282 ',http://zoo-hst.s3.amazonaws.com/10000278.jpg,10000278,-999,b'AEGIS ',...,False,False,False,False,False,False,False,False,b'original_initial ',False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106687,169934,9010781865,AHZ700029z,53.098407,-27.766867,b'120 ',http://zoo-hst.s3.amazonaws.com/9010781865.jpg,9010781865,0,b'GOODS-S',...,False,False,False,False,False,False,False,False,b'fake_agn_v2 ',False
106688,169935,9010781871,AHZ70002a0,53.098407,-27.766867,b'120 ',http://zoo-hst.s3.amazonaws.com/9010781871.jpg,9010781871,0,b'GOODS-S',...,False,False,False,False,False,False,False,False,b'fake_agn_v2 ',False
106689,169936,9010781872,AHZ70002a1,53.098407,-27.766867,b'120 ',http://zoo-hst.s3.amazonaws.com/9010781872.jpg,9010781872,0,b'GOODS-S',...,False,False,False,False,False,False,False,False,b'fake_agn_v2 ',False
106690,169939,9010781875,AHZ70002a4,53.098407,-27.766867,b'120 ',http://zoo-hst.s3.amazonaws.com/9010781875.jpg,9010781875,0,b'GOODS-S',...,False,False,False,False,False,False,False,False,b'fake_agn_v2 ',False


In [85]:
df['location'][0]

'http://zoo-hst.s3.amazonaws.com/10000189.jpg'

In [86]:
df['location'].sample(1).squeeze()

'http://zoo-hst.s3.amazonaws.com/20037040.jpg'

In [87]:
len(df)

106692

In [88]:
gzh_urls_to_download_loc = '/nvme1/scratch/walml/repos/pytorch-galaxy-datasets/hidden_data/gzh_urls.txt'

In [89]:
with open(gzh_urls_to_download_loc, 'w') as f:
    f.writelines('\n'.join(list(df['location'])))

In [90]:
print(f'cat {gzh_urls_to_download_loc} | xargs -n100 -P8 wget --continue --directory-prefix /nvme1/scratch/walml/repos/pytorch-galaxy-datasets/roots/gzh/images')

# batch size of 100 (urls), 8 processes

cat /nvme1/scratch/walml/repos/pytorch-galaxy-datasets/hidden_data/gzh_urls.txt | xargs -n100 -P8 wget --continue --directory-prefix /nvme1/scratch/walml/repos/pytorch-galaxy-datasets/roots/gzh/images


In [91]:
label_df = pd.read_csv('/home/walml/repos/pytorch-galaxy-datasets/hidden_data/gz_hubble_main.csv')
label_df

,zooniverse_id,survey_id,Table,RA,DEC,Z_BEST,GZ_MU_I,imaging,correction_type,total_count,...,t17_clumps_symmetrical_a02_no_fraction,t17_clumps_symmetrical_a02_no_weighted_fraction,t17_clumps_symmetrical_total_count,t17_clumps_symmetrical_total_weight,t18_clumps_embedded_a01_yes_fraction,t18_clumps_embedded_a01_yes_weighted_fraction,t18_clumps_embedded_a02_no_fraction,t18_clumps_embedded_a02_no_weighted_fraction,t18_clumps_embedded_total_count,t18_clumps_embedded_total_weight
0,AGZ0007xmh,1,hubble,188.897318,62.200334,0.090,21.305056,GOODS-N-FULLDEPTH,2,40,...,0.333333,0.333333,3,3.0,0.666667,0.666667,0.333333,0.333333,3,3.0
1,AGZ0007xmi,4,hubble,188.901464,62.200705,999.000,23.121389,GOODS-N-FULLDEPTH,4,41,...,0.769231,0.769231,13,13.0,0.461538,0.461538,0.538462,0.538462,13,13.0
2,AGZ0007xmj,6,hubble,188.901567,62.201482,0.270,21.161961,GOODS-N-FULLDEPTH,2,40,...,0.500000,0.500000,6,6.0,0.666667,0.666667,0.333333,0.333333,6,6.0
3,AGZ0007xmk,13,hubble,188.905098,62.199973,999.000,23.676237,GOODS-N-FULLDEPTH,4,40,...,0.500000,0.500000,2,2.0,0.000000,0.000000,1.000000,1.000000,2,2.0
4,AGZ0007xml,16,hubble,188.906262,62.198074,999.000,21.821812,GOODS-N-FULLDEPTH,4,41,...,0.666667,0.666667,3,3.0,0.333333,0.333333,0.666667,0.666667,3,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113700,AHZ40008nu,90105528,hubble,52.873002,-27.580397,0.332,19.472644,GEMS,0,173,...,0.812000,0.812000,16,16.0,0.750000,0.750000,0.250000,0.250000,16,16.0
113701,AHZ40008nv,90105532,hubble,53.252911,-27.645872,999.000,19.235010,GEMS,4,87,...,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.0
113702,AHZ40008nw,90105589,hubble,52.982787,-27.578840,1.339,19.372154,GEMS,3,148,...,0.900000,0.900000,10,10.0,0.700000,0.700000,0.300000,0.300000,10,10.0
113703,AHZ40008nx,90107105,hubble,53.189682,-27.948248,999.000,19.059707,GEMS,4,31,...,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,0.000000,0,0.0


In [92]:
df = pd.merge(label_df, df[['zooniverse_id', 'location']], on='zooniverse_id', how='inner')

Also modify the catalog to follow my schema/label_cols convention

In [93]:
df_before_rename = df.copy()
[col for col in df_before_rename.columns.values if 'limit' not in col and 'weight' not in col]

['zooniverse_id',
 'survey_id',
 'Table',
 'RA',
 'DEC',
 'Z_BEST',
 'GZ_MU_I',
 'imaging',
 'correction_type',
 'total_count',
 't01_smooth_or_features_a01_smooth_fraction',
 't01_smooth_or_features_a01_smooth_debiased_fraction',
 't01_smooth_or_features_a01_smooth_best_fraction',
 't01_smooth_or_features_a01_smooth_flag',
 't01_smooth_or_features_a02_features_or_disk_fraction',
 't01_smooth_or_features_a02_features_or_disk_debiased_fraction',
 't01_smooth_or_features_a02_features_or_disk_best_fraction',
 't01_smooth_or_features_a02_features_or_disk_flag',
 't01_smooth_or_features_a03_star_or_artifact_fraction',
 't01_smooth_or_features_a03_star_or_artifact_flag',
 't01_smooth_or_features_total_count',
 't02_edgeon_a01_yes_fraction',
 't02_edgeon_a02_no_fraction',
 't02_edgeon_total_count',
 't03_bar_a01_bar_fraction',
 't03_bar_a02_no_bar_fraction',
 't03_bar_total_count',
 't04_spiral_a01_spiral_fraction',
 't04_spiral_a02_no_spiral_fraction',
 't04_spiral_total_count',
 't05_bulge_

In [94]:

# rename the questions (for count columns)
question_renamer = {
    't01_smooth_or_features': 'smooth-or-featured',
    't02_edgeon': 'disk-edge-on',
    't03_bar': 'bar',
    't04_spiral': 'has-spiral-arms',
    't05_bulge_prominence': 'bulge-size',
    # t06 is odd
    't07_rounded': 'how-rounded',
    't09_bulge_shape': 'bulge-shape',  # not in candels
    't10_arms_winding': 'spiral-winding',
    't11_arms_number': 'spiral-arm-count',
    't12_clumpy': 'clumpy-appearance',
    't16_clumps_count': 'clump-count',
    't17_clumps_symmetrical': 'galaxy-symmetrical',
    't18_clumps_embedded': 'clumps-embedded-larger-object'
}

# define the new schema
# this has been moved to the prepared dataset itself - see prepared_datasets/candels
from pytorch_galaxy_datasets.prepared_datasets import candels



In [95]:




# rename the _frac's according to new schema
column_renamer = {
    't01_smooth_or_features_a01_smooth_fraction': 'smooth-or-featured_smooth_fraction',
    't01_smooth_or_features_a02_features_or_disk_fraction': 'smooth-or-featured_features_fraction',
    # etc
    't01_smooth_or_features_a03_star_or_artifact_fraction': 'smooth-or-featured_artifact_fraction',
    't02_edgeon_a01_yes_fraction': 'disk-edge-on_yes_fraction',
    't02_edgeon_a02_no_fraction': 'disk-edge-on_no_fraction',
    't03_bar_a01_bar_fraction': 'bar_yes_fraction',
    't03_bar_a02_no_bar_fraction': 'bar_no_fraction',
    't04_spiral_a01_spiral_fraction': 'has-spiral-arms_yes_fraction',
    't04_spiral_a02_no_spiral_fraction': 'has-spiral-arms_no_fraction',
    't05_bulge_prominence_a01_no_bulge_fraction': 'bulge-size_none_fraction',
    't05_bulge_prominence_a02_just_noticeable_fraction': 'bulge-size_just-noticeable_fraction',
    't05_bulge_prominence_a03_obvious_fraction': 'bulge-size_obvious_fraction',
    't05_bulge_prominence_a04_dominant_fraction': 'bulge-size_dominant_fraction',
    't07_rounded_a01_completely_round_fraction': 'how-rounded_completely_fraction',
    't07_rounded_a02_in_between_fraction': 'how-rounded_in-between_fraction',
    't07_rounded_a03_cigar_shaped_fraction': 'how-rounded_cigar-shaped_fraction',
    't09_bulge_shape_a01_rounded_fraction': 'bulge-shape_rounded_fraction',
    't09_bulge_shape_a02_boxy_fraction': 'bulge-shape_boxy_fraction',
    't09_bulge_shape_a03_no_bulge_fraction': 'bulge-shape_none_fraction',
    't10_arms_winding_a01_tight_fraction': 'spiral-winding_tight_fraction',
    't10_arms_winding_a02_medium_fraction': 'spiral-winding_medium_fraction',
    't10_arms_winding_a03_loose_fraction': 'spiral-winding_loose_fraction',
    't11_arms_number_a01_1_fraction': 'spiral-arm-count_1_fraction',
    't11_arms_number_a02_2_fraction': 'spiral-arm-count_2_fraction',
    't11_arms_number_a03_3_fraction': 'spiral-arm-count_3_fraction',
    't11_arms_number_a04_4_fraction': 'spiral-arm-count_4_fraction',
    't11_arms_number_a05_more_than_4_fraction': 'spiral-arm-count_5-plus_fraction',
    't11_arms_number_a06_cant_tell_fraction': 'spiral-arm-count_cant-tell_fraction',
    't12_clumpy_a01_yes_fraction': 'clumpy-appearance_yes_fraction',
    't12_clumpy_a02_no_fraction': 'clumpy-appearance_no_fraction',
    't16_clumps_count_a01_1_fraction': 'clump-count_1_fraction',
    't16_clumps_count_a02_2_fraction': 'clump-count_2_fraction',
    't16_clumps_count_a03_3_fraction': 'clump-count_3_fraction',
    't16_clumps_count_a04_4_fraction': 'clump-count_4_fraction',
    't16_clumps_count_a05_more_than_4_fraction': 'clump-count_5-plus_fraction',
    't16_clumps_count_a06_cant_tell_fraction': 'clump-count_cant-tell_fraction',
    't17_clumps_symmetrical_a01_yes_fraction': 'galaxy-symmetrical_yes_fraction',
    't17_clumps_symmetrical_a02_no_fraction': 'galaxy-symmetrical_no_fraction',
    't18_clumps_embedded_a01_yes_fraction': 'clumps-embedded-larger-object_yes_fraction',
    't18_clumps_embedded_a02_no_fraction': 'clumps-embedded-larger-object_no_fraction'
}

# rename to ortho style

df = df_before_rename.rename(columns=column_renamer)

for col in df.columns.values:
    if 'weight' in col:
        del df[col]

[x for x in df.columns.values if '_frac' in x and 'debiased' not in x and 'best' not in x]

['smooth-or-featured_smooth_fraction',
 'smooth-or-featured_features_fraction',
 'smooth-or-featured_artifact_fraction',
 'disk-edge-on_yes_fraction',
 'disk-edge-on_no_fraction',
 'bar_yes_fraction',
 'bar_no_fraction',
 'has-spiral-arms_yes_fraction',
 'has-spiral-arms_no_fraction',
 'bulge-size_none_fraction',
 'bulge-size_just-noticeable_fraction',
 'bulge-size_obvious_fraction',
 'bulge-size_dominant_fraction',
 't06_odd_a01_yes_fraction',
 't06_odd_a02_no_fraction',
 'how-rounded_completely_fraction',
 'how-rounded_in-between_fraction',
 'how-rounded_cigar-shaped_fraction',
 't08_odd_feature_a01_ring_fraction',
 't08_odd_feature_a02_lens_or_arc_fraction',
 't08_odd_feature_a03_disturbed_fraction',
 't08_odd_feature_a04_irregular_fraction',
 't08_odd_feature_a05_other_fraction',
 't08_odd_feature_a06_merger_fraction',
 't08_odd_feature_a07_dust_lane_fraction',
 'bulge-shape_rounded_fraction',
 'bulge-shape_boxy_fraction',
 'bulge-shape_none_fraction',
 'spiral-winding_tight_fracti

In [96]:


new_questions = question_renamer.values()
# assert set(new_questions) == set(candels.candels_pairs.keys())
# set(new_questions) - set(candels.candels_pairs.keys())


In [97]:
# naively this is okay, but due to the rare rounding issue below, I'm not sure the count column is correct
# therefore

# also rename the count columns
# relies on implicitly ordered dict, fine in python 3.7
old_count_cols = [x + '_total_count' for x in question_renamer.keys()]
new_count_cols = [x + '_total-votes' for x in question_renamer.values()]

count_renamer = dict(zip(old_count_cols, new_count_cols))
df = df.rename(columns=count_renamer)

In [98]:
from pytorch_galaxy_datasets.prepared_datasets import hubble

# calculate the answer counts according to new schema
for question, answers in hubble.hubble_pairs.items():
    for answer in answers:
        counts = df[question + '_total-votes'] * df[question + answer + '_fraction']

        # check that they are all integers - they aren't, and I've asked Brooke about this
        # rounded_counts = np.around(counts)
        # rounded_count_matches_counts = np.isclose(df[question + answer], np.around(df[question + answer]))
        # if not rounded_count_matches_counts.all():
        #     print(question, answer, rounded_count_matches_counts.mean())
        #     raise ValueError(counts[~rounded_count_matches_counts])

        # temporarily, just round them
        rounded_counts = np.around(counts)

        df[question + answer] = rounded_counts

    answer_cols = [question + answer for answer in answers]
    total_from_new_counts = df[answer_cols].sum(axis=1)
    # total_from_raw_counts = df[question + '_total-votes']
    # totals_match = total_from_new_counts == total_from_raw_counts
    # if not totals_match.all():
    #     print(question, answer, totals_match.mean())
    #     raise ValueError(total_from_raw_counts[~totals_match], total_from_new_counts[~totals_match])
    df[question + '_total-votes'] = total_from_new_counts  # for now, also update the total-votes from the rounded counts. Ensures dirichlet will be happy.


        

In [99]:
# row = df_before_rename.iloc[20221]
# row['t03_how_many_clumps_a0_1_frac'], row['t03_how_many_clumps_count'], row['t03_how_many_clumps_a0_1_frac'] * row['t03_how_many_clumps_count']

In [100]:
df.columns.values

array(['zooniverse_id', 'survey_id', 'Table', 'RA', 'DEC', 'Z_BEST',
       'GZ_MU_I', 'imaging', 'correction_type', 'total_count',
       'smooth-or-featured_smooth_fraction',
       't01_smooth_or_features_a01_smooth_debiased_fraction',
       't01_smooth_or_features_a01_smooth_lower_limit',
       't01_smooth_or_features_a01_smooth_upper_limit',
       't01_smooth_or_features_a01_smooth_best_fraction',
       't01_smooth_or_features_a01_smooth_flag',
       'smooth-or-featured_features_fraction',
       't01_smooth_or_features_a02_features_or_disk_debiased_fraction',
       't01_smooth_or_features_a02_features_or_disk_lower_limit',
       't01_smooth_or_features_a02_features_or_disk_upper_limit',
       't01_smooth_or_features_a02_features_or_disk_best_fraction',
       't01_smooth_or_features_a02_features_or_disk_flag',
       'smooth-or-featured_artifact_fraction',
       't01_smooth_or_features_a03_star_or_artifact_flag',
       'smooth-or-featured_total-votes', 'disk-edge-on_yes

In [101]:
df

,zooniverse_id,survey_id,Table,RA,DEC,Z_BEST,GZ_MU_I,imaging,correction_type,total_count,...,spiral-arm-count_5-plus,spiral-arm-count_cant-tell,bulge-size_none,bulge-size_just-noticeable,bulge-size_obvious,bulge-size_dominant,galaxy-symmetrical_yes,galaxy-symmetrical_no,clumps-embedded-larger-object_yes,clumps-embedded-larger-object_no
0,AHZ1000001,10000189,hubble,214.235824,52.396332,999.000,18.514021,AEGIS,4,412,...,0.0,0.0,0.0,0.0,7.0,8.0,5.0,0.0,5.0,0.0
1,AHZ1000002,10000215,hubble,214.328535,52.407050,999.000,19.934329,AEGIS,4,429,...,0.0,0.0,1.0,6.0,4.0,0.0,1.0,2.0,2.0,1.0
2,AHZ1000003,10000235,hubble,214.130396,52.452309,999.000,20.926447,AEGIS,4,728,...,0.0,20.0,8.0,18.0,4.0,0.0,25.0,16.0,32.0,9.0
3,AHZ1000004,10000249,hubble,214.225495,52.408517,999.000,20.968599,AEGIS,4,490,...,0.0,2.0,9.0,10.0,6.0,0.0,2.0,3.0,2.0,3.0
4,AHZ1000005,10000278,hubble,214.257599,52.450259,999.000,21.076458,AEGIS,4,513,...,0.0,0.0,22.0,0.0,0.0,0.0,3.0,6.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100862,AHZ40008nt,90105522,hubble,53.208246,-27.577137,0.243,19.708874,GEMS,2,257,...,1.0,6.0,0.0,6.0,10.0,4.0,0.0,4.0,3.0,1.0
100863,AHZ40008nu,90105528,hubble,52.873002,-27.580397,0.332,19.472644,GEMS,0,173,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,13.0,12.0,4.0
100864,AHZ40008nw,90105589,hubble,52.982787,-27.578840,1.339,19.372154,GEMS,3,148,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0,7.0,3.0
100865,AHZ40008nx,90107105,hubble,53.189682,-27.948248,999.000,19.059707,GEMS,4,31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Some final column tidying

In [102]:
cols_to_drop = [
    'Table', 'total_count', 'GZ_MU_I']  # remove location_standard to not reveal the internal urls (causing AWS bills)

for col in cols_to_drop:
    del df[col]

In [103]:
df['id_str'] = df['survey_id'].astype(str)

In [104]:
df = df.rename(columns={
    'RA': 'ra',
    'DEC': 'dec',
    'Z_BEST': 'redshift'
})

In [105]:
df['filename'] = df['id_str'] + '.jpg'

In [106]:
df['filename'].iloc[0]

'10000189.jpg'

In [107]:
temp = '/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/' + df['filename']

import os
assert os.path.isfile(temp[0])

is_file = [os.path.isfile(loc) for loc in temp]
np.mean(is_file), len(is_file) -  np.sum(is_file)

(0.9994943836933784, 51)

In [108]:
df = df[is_file].reset_index(drop=True)  # 51 are missing, drop these

In [109]:
from PIL import Image

temp = '/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/' + df['filename']


for loc in temp:
    if not Image.open(loc).size == (424, 424):
        print(loc)
        # break

/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/90006989.jpg
/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/90006947.jpg
/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/90006800.jpg
/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/90014714.jpg
/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/90008337.jpg
/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/90017422.jpg
/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/90016039.jpg
/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/90012194.jpg
/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/90015145.jpg
/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/90011309.jpg
/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/90016550.jpg
/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/90071123.jpg
/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/90015324.jpg
/home/walml/repos/pytorch

In [110]:

is_correct_shape_pil = np.array([Image.open(loc).size == (424, 424) for loc in temp])
np.sum(is_correct_shape_pil)

100790

In [111]:

# same result (26 bad) but much slower
# is_correct_shape_np = [np.array(Image.open(loc)).shape == (424, 424, 3) for loc in temp]

In [112]:
df = df[is_correct_shape_pil].reset_index(drop=True)  # 51 are missing, drop these

In [113]:


temp = '/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/images/' + df['filename']
is_correct_shape_pil = np.array([Image.open(loc).size == (424, 424) for loc in temp])
assert all(is_correct_shape_pil)

In [114]:
# rename to ortho style

normal_answer_cols = []
for question, answers in hubble.hubble_pairs.items():
    for answer in answers:
        normal_answer_cols.append(question + answer)
        normal_answer_cols.append(question + answer + '_fraction')
        normal_answer_cols.append(question + '_total-votes')


ortho_answer_cols = []
for question, answers in hubble.hubble_ortho_pairs.items():
    for answer in answers:
        ortho_answer_cols.append(question + answer)
        ortho_answer_cols.append(question + answer + '_fraction')
        ortho_answer_cols.append(question + '_total-votes')

df = df.rename(columns=dict(zip(normal_answer_cols, ortho_answer_cols)))

df.columns.values

array(['zooniverse_id', 'survey_id', 'ra', 'dec', 'redshift', 'imaging',
       'correction_type', 'smooth-or-featured-hubble_smooth_fraction',
       't01_smooth_or_features_a01_smooth_debiased_fraction',
       't01_smooth_or_features_a01_smooth_lower_limit',
       't01_smooth_or_features_a01_smooth_upper_limit',
       't01_smooth_or_features_a01_smooth_best_fraction',
       't01_smooth_or_features_a01_smooth_flag',
       'smooth-or-featured-hubble_features_fraction',
       't01_smooth_or_features_a02_features_or_disk_debiased_fraction',
       't01_smooth_or_features_a02_features_or_disk_lower_limit',
       't01_smooth_or_features_a02_features_or_disk_upper_limit',
       't01_smooth_or_features_a02_features_or_disk_best_fraction',
       't01_smooth_or_features_a02_features_or_disk_flag',
       'smooth-or-featured-hubble_artifact_fraction',
       't01_smooth_or_features_a03_star_or_artifact_flag',
       'smooth-or-featured-hubble_total-votes',
       'disk-edge-on-hubble_y

Now split

In [115]:
train_catalog, test_catalog = train_test_split(df, test_size=0.2, random_state=42)

In [116]:
train_loc = '/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/hubble_ortho_train_catalog.parquet'
test_loc = '/home/walml/repos/pytorch-galaxy-datasets/roots/hubble/hubble_ortho_test_catalog.parquet'

In [117]:
# train_catalog.to_parquet(train_loc, index=False)
# test_catalog.to_parquet(test_loc, index=False)

In [118]:
images_loc = '/nvme1/scratch/walml/repos/pytorch-galaxy-datasets/roots/hubble/hubble_images.tar.gz'

In [119]:
for loc in [
    train_loc,
    test_loc,
    images_loc
]:
    # print hash
    with open(loc, 'rb') as f:
        md5_checksum = hashlib.md5(f.read()).hexdigest()

    print(md5_checksum)

c6cb821f7ebefb583dc74488cf7bfc5f
05e01ed822b34400f32977280eebec87
b6fed2463bb2d17ddb8302f6b060534a
